# Q-learning and deep Q-learning demonstration

In [ ]:
## Import Libraries
import numpy as np
import gym
import random
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [ ]:
## Initialize the CliffWalking environment
env = gym.make('CliffWalking-v0')

In [ ]:
## Set up Q-learning parameters
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

num_episodes = 5000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

rewards_all_episodes = []

In [ ]:
## Q-Learning algorithm
for episode in range(num_episodes):
    state = env.reset()
    done = False
    rewards_current_episode = 0

    for step in range(max_steps_per_episode):
        
        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:])
        else:
            action = env.action_space.sample()
        
        new_state, reward, done, info = env.step(action)
        
        # Update Q-table
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        state = new_state
        rewards_current_episode += reward
        
        if done:
            break
            
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    
    rewards_all_episodes.append(rewards_current_episode)

In [ ]:
## Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)
count = 1000

print("Average reward per thousand episodes")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

In [ ]:
## Visualize the Q-table
print("Q-table")
print(q_table)

In [ ]:
## Plotting the results
plt.plot(range(num_episodes), rewards_all_episodes)
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.title('Reward vs Episode')
plt.show()

In [ ]:
## Function to visualize the agent's movement
def visualize_agent_path(env, q_table):
    # Create a grid to visualize
    grid_size = (4, 12)  # CliffWalking-v0 has a 4x12 grid
    actions = ['U', 'D', 'L', 'R']  # Up, Down, Left, Right

    def print_grid(path=None):
        grid = np.zeros(grid_size)
        if path:
            for (i, j, action) in path:
                grid[i, j] = 1
        plt.imshow(grid, cmap='viridis', interpolation='none')
        plt.title('Agent Path')
        plt.colorbar()
        plt.show()

    state = env.reset()
    path = []
    done = False

    for _ in range(max_steps_per_episode):
        action = np.argmax(q_table[state,:])
        row, col = divmod(state, grid_size[1])
        path.append((row, col, actions[action]))
        next_state, _, done, _ = env.step(action)
        state = next_state
        if done:
            break

    print_grid(path)

In [ ]:
## Run the visualization
visualize_agent_path(env, q_table)